# Enhancing RAG with Neo4j Knowledge Graph


Import modules and instantiate connections and models

In [2]:
import snowflake.connector
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import config
import graph


embedding_model = HuggingFaceEmbeddings(
    model_name=config.EMBEDDING_MODEL_CHECKPOINT,
    model_kwargs=dict(trust_remote_code=True, revision=config.EMBEDDING_MODEL_HASH)
)

vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    search_type='vector'
)

In [4]:
question = 'What does the FDP want to do?'
# filter = {'category': {"$eq": "summary"}}
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

page_content='FDP-Fraktionschef Christian Dürr pocht dagegen auf Sparsamkeit etwa bei der Entwicklungshilfe. „Die Haltung der FDP hat sich nicht geändert: Wir müssen richtig priorisieren, etwa bei Verteidigung, Bildung und Infrastruktur“, sagte er den Funke-Zeitungen.' metadata={'position': 26, 'uid': 'Chunk:5OsLGWEtSd6M', 'category': 'paragraph', 'section': 6}
page_content='FDP-Generalsekretär Bijan Djir-Sarai nahm Lindner in Schutz. Deutschland brauche einen verfassungskonformen Haushalt, sagte er der Deutschen Presse-Agentur. Dass Lindner einzelne Vorhaben einer unabhängigen Prüfung unterzogen habe, sei deshalb vollkommen richtig.' metadata={'position': 13, 'uid': 'Chunk:XNmTCGfaTGWz', 'category': 'paragraph', 'section': 3}
page_content='Auf Kritik vor allem der SPD erwiderte Lindner am Freitagabend bei einem Wahlkampftermin in Potsdam: „Ich bin erstaunt, denn auch Sozialdemokraten in Partei und Bundestag wissen ja, dass es drei Prüfaufträge gab und keine politische Verabredung.“ Di

In [5]:
vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    search_type='hybrid'
)
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

page_content='It is no use appealing to the better nature of any of these figures, however. It is what they do. The prime minister is right to demand more social responsibility from the tech companies, just as he is right to want the full force of the law to be used to keep the peace and protect the police and, at the moment, Muslim people from harm.' metadata={'position': 5, 'uid': 'Chunk:wOCntvk5TOem', 'category': 'paragraph', 'section': 1}
page_content='FDP-Fraktionschef Christian Dürr pocht dagegen auf Sparsamkeit etwa bei der Entwicklungshilfe. „Die Haltung der FDP hat sich nicht geändert: Wir müssen richtig priorisieren, etwa bei Verteidigung, Bildung und Infrastruktur“, sagte er den Funke-Zeitungen.' metadata={'position': 26, 'uid': 'Chunk:5OsLGWEtSd6M', 'category': 'paragraph', 'section': 6}
page_content="'I know he wants to play important moments, yeah. But the other ones [do] too. 18, 19 players as well want to play the big games." metadata={'position': 8, 'uid': 'Chunk:RFWH6

In [6]:
retrieval_query=(
    "WITH node as chunk, score "
    "MATCH (chunk)<-[:CONTAINS]-(a)<-[:PUBLISHED]-(s) "
    "WITH chunk, score, a, s "
    "RETURN 'Title: ' + a.title + '\\nText: ' + chunk.text as text, score, "
    "chunk{.position, .section, .category, date: a.publishing_date, url:a.url, source: s.name} as metadata"
)
print(retrieval_query)
vector_index = Neo4jVector.from_existing_index(
    embedding_model,
    url=graph.URI,
    username=graph.USERNAME,
    password=graph.PASSWORD,
    index_name='chunkEmbedding',
    keyword_index_name='chunkText',
    retrieval_query=retrieval_query
)
retrieved_docs = vector_index.similarity_search(question, k=5, filter=None)
for doc in retrieved_docs:
    print(doc)

WITH node as chunk, score MATCH (chunk)<-[:CONTAINS]-(a)<-[:PUBLISHED]-(s) WITH chunk, score, a, s RETURN 'Title: ' + a.title + '\nText: ' + chunk.text as text, score, chunk{.position, .section, .category, date: a.publishing_date, url:a.url, source: s.name} as metadata
page_content='Title: Ampel im Haushaltsstreit massiv in der Kritik: „Kann nicht seriös regieren“\nText: FDP-Fraktionschef Christian Dürr pocht dagegen auf Sparsamkeit etwa bei der Entwicklungshilfe. „Die Haltung der FDP hat sich nicht geändert: Wir müssen richtig priorisieren, etwa bei Verteidigung, Bildung und Infrastruktur“, sagte er den Funke-Zeitungen.' metadata={'position': 26, 'category': 'paragraph', 'source': 'Focus Online', 'date': neo4j.time.DateTime(2024, 8, 3, 12, 50, 8, 0, tzinfo=<UTC>), 'section': 6, 'url': 'https://www.focus.de/politik/deutschland/wegen-haushaltsstreit-kann-nicht-serioes-regieren-so-hart-steht-die-ampel-derzeit-in-der-kritik_id_260194099.html'}
page_content='Title: Ampel im Haushaltsstreit

In [7]:
import snowflake.connector
from langchain.chains import RetrievalQA

from llm import Cortex

snowflake_connection = snowflake.connector.connect(**config.SNOWFLAKE_CONNECTION_PARAMS)
model = Cortex(connection=snowflake_connection, model=config.CHAT_MODEL)

vector_qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type='stuff',
    retriever=vector_index.as_retriever()
)

In [8]:
vector_qa.run(question)

/home/djordje/.virtualenvs/bbuzz24/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Complete() is experimental since 1.0.12. Do not use it in production. 


' The FDP wants to prioritize spending on areas such as defense, education, and infrastructure, and maintain a constitutional and economically sound budget. They reject the idea of suspending the debt brake and its fundamental reform.'

## Generate a Cypher query

In [11]:
from langchain.chains import GraphCypherQAChain

db = graph.NewsGraphClient()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = model,
    qa_llm = model,
    graph=db.graph,
    verbose=True
)


In [12]:
cypher_chain.run(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
 MATCH (fdp:Organization)-[:MENTIONS]->(p:Person)
WHERE fdp.name = 'FDP'
RETURN p.name AS person_name, fdp.name AS organization_name
Full Context:
[]

> Finished chain.


" I don't know the answer based on the provided information."